In [13]:
setwd("/data/home/stoecker/fdi_plantpathologycoop_mount/cbi_ipoolseq_transposon/ipoolseq-pipeline/notebooks/")

#pseudo snakemake variables added here 
snakemake_params_exp = "Exp1"
snakemake_params_dir = "Transposon.v2"
snakemake_params_version = "0.2"
snakemake_input_ann = "../cfg/Transposon.v2/annotation.gff3.gz"
snakemake_input_essential = "../cfg/Transposon.v2/essential.tab"

In [6]:
# Set "eval=TRUE" above to run this chunk interactively in R Studio,
# and adjust the input and output files to point to the sample you
# want to test with
#design <- "Transposon"
#replicate <- "r77647"

design <- snakemake_params_dir
replicate <- snakemake_params_exp

setClass("SnakemakeStub", representation(input = "list", params = "list", output = "list"))
snakemake <- new("SnakemakeStub",
                 input=list(ann=paste("..", "cfg", design, "annotation.gff3.gz", sep="/"),
                            essential=paste("..", "cfg", design, "essential.tab", sep="/"),
                            isites_in=paste("..", "data", design, paste0(replicate, "-in.isites.gff3.gz"), sep="/"),
                            isites_out=paste("..", "data", design, paste0(replicate, "-out.isites.gff3.gz"), sep="/"),
                            pool_in_5p=paste("..", "data", design, paste0(replicate, "-in+5p.count.tab"), sep="/"),
                            pool_in_3p=paste("..", "data", design, paste0(replicate, "-in+3p.count.tab"), sep="/"),
                            pool_out_5p=paste("..", "data", design, paste0(replicate, "-out+5p.count.tab"), sep="/"),
                            pool_out_3p=paste("..", "data", design, paste0(replicate, "-out+3p.count.tab"), sep="/"),
                            stats_in_5p=paste("..", "data", design, paste0(replicate, "-in+5p.stats.tab"), sep="/"),
                            stats_in_3p=paste("..", "data", design, paste0(replicate, "-in+3p.stats.tab"), sep="/"),
                            stats_out_5p=paste("..", "data", design, paste0(replicate, "-out+5p.stats.tab"), sep="/"),
                            stats_out_3p=paste("..", "data", design, paste0(replicate, "-out+3p.stats.tab"), sep="/"),
                            trumicount_pdf_in_5p=paste("..", "data", design, paste0(replicate, "-in+5p.count.pdf"), sep="/"),
                            trumicount_pdf_in_3p=paste("..", "data", design, paste0(replicate, "-in+3p.count.pdf"), sep="/"),
                            trumicount_pdf_out_5p=paste("..", "data", design, paste0(replicate, "-out+5p.count.pdf"), sep="/"),
                            trumicount_pdf_out_3p=paste("..", "data", design, paste0(replicate, "-out+3p.count.pdf"), sep="/"),
                            fastqc_html_in_5p_r1=paste("..", "data", design, paste0(replicate, "-in+5p.fastqc.1.html"), sep="/"),
                            fastqc_html_in_5p_r2=paste("..", "data", design, paste0(replicate, "-in+5p.fastqc.2.html"), sep="/"),
                            fastqc_html_in_3p_r1=paste("..", "data", design, paste0(replicate, "-in+3p.fastqc.1.html"), sep="/"),
                            fastqc_html_in_3p_r2=paste("..", "data", design, paste0(replicate, "-in+3p.fastqc.2.html"), sep="/"),
                            fastqc_html_out_5p_r1=paste("..", "data", design, paste0(replicate, "-out+5p.fastqc.1.html"), sep="/"),
                            fastqc_html_out_5p_r2=paste("..", "data", design, paste0(replicate, "-out+5p.fastqc.2.html"), sep="/"),
                            fastqc_html_out_3p_r1=paste("..", "data", design, paste0(replicate, "-out+3p.fastqc.1.html"), sep="/"),
                            fastqc_html_out_3p_r2=paste("..", "data", design, paste0(replicate, "-out+3p.fastqc.2.html"), sep="/")),
                 params=list(version=local({f <- file("../VERSION", "r"); v <- readLines(f, n=1); close(f); v}),
                             dir=design, exp=replicate),
                 output=list(table=paste("..", "data", design, paste0(replicate, "dv.tab"), sep="/")))

In [7]:
snakemake

An object of class "SnakemakeStub"
Slot "input":
$ann
[1] "../cfg/Transposon.v2/annotation.gff3.gz"

$essential
[1] "../cfg/Transposon.v2/essential.tab"

$isites_in
[1] "../data/Transposon.v2/Exp1-in.isites.gff3.gz"

$isites_out
[1] "../data/Transposon.v2/Exp1-out.isites.gff3.gz"

$pool_in_5p
[1] "../data/Transposon.v2/Exp1-in+5p.count.tab"

$pool_in_3p
[1] "../data/Transposon.v2/Exp1-in+3p.count.tab"

$pool_out_5p
[1] "../data/Transposon.v2/Exp1-out+5p.count.tab"

$pool_out_3p
[1] "../data/Transposon.v2/Exp1-out+3p.count.tab"

$stats_in_5p
[1] "../data/Transposon.v2/Exp1-in+5p.stats.tab"

$stats_in_3p
[1] "../data/Transposon.v2/Exp1-in+3p.stats.tab"

$stats_out_5p
[1] "../data/Transposon.v2/Exp1-out+5p.stats.tab"

$stats_out_3p
[1] "../data/Transposon.v2/Exp1-out+3p.stats.tab"

$trumicount_pdf_in_5p
[1] "../data/Transposon.v2/Exp1-in+5p.count.pdf"

$trumicount_pdf_in_3p
[1] "../data/Transposon.v2/Exp1-in+3p.count.pdf"

$trumicount_pdf_out_5p
[1] "../data/Transposon.v2/Exp1-out+5p.coun

In [10]:
# Load output libraries
library(knitr)
library(rmdformats)

# Load libraries
library(data.table)
library(DT)
library(rtracklayer)
library(plotly)
source("../scripts/ipoolseq.model.R")

# Output '-' for NA values
options(knitr.kable.NA = '-')

# Set FDR to 5%
PLOT.FDR.THRESHOLD <- 0.05

# Convert a p-value into a significance marker
sig.stars <- function(p) {
  pp <- ifelse((p > 0) & (p < 1), p, 0.5)
  ifelse(!is.finite(p) | (p < 0) | (p > 1), "?",
         ifelse(p > 0.1, " ",
                ifelse(p > 0.05, "+",
                       ifelse(p == 0, "*****",
                              strrep("*", pmin(ceiling(-log10(pp)*(1+.Machine$double.eps))-1, 5))))))
}


In [11]:
# Create a Markdown link
make_link <- function(path, text) {
  paste0('[', if(missing(text)) path else text, '](', URLencode(path, reserved=TRUE), ')')
}

# Correlation with zero intercept
cor.intercept0 <- function(x, y) {
  i <- !is.na(x) & !is.na(y)
  k <- sum(x[i] * y[i]) / sum(x[i]**2)
  r.sq <- 1 - sum((y[i] - k*x[i])**2) / sum((y[i] - mean(y[i]))**2)
  sqrt(r.sq)
}


In [14]:
# Loading gene annotation
message("Loading gene annotations from ", snakemake_input_ann)
annotations <- readGFFAsGRanges(snakemake_input_ann, version=3)
genes <- annotations[annotations$type=="gene"]

# Mark essential genes with Essential=TRUE
essential <- read.table(snakemake_input_essential, header=TRUE)
genes$Essential <- mcols(genes)[[colnames(essential)[1]]] %in% essential[[1]]

# Load list of mutants, find assumed neutral set
message("Loading insertion sites from ", snakemake@input$isites_in)
mutants <- readGFFAsGRanges(snakemake@input$isites_in, version=3)
# Assign mutants to genes
local({
  hits <- findOverlaps(mutants, genes, ignore.strand=TRUE)
  mutants$GeneIndex <<- NA_integer_
  mutants$GeneIndex[queryHits(hits)] <<- subjectHits(hits)
})
# Mark mutants not assigned to genes as neutral
mutants$Neutral <- ifelse(is.na(mutants$GeneIndex), 1, 0)

# Find neutral mutants!
NEUTRAL <- mutants$Name[mutants$Neutral == 1]


Loading gene annotations from ../cfg/Transposon.v2/annotation.gff3.gz

Loading insertion sites from ../data/Transposon.v2/Exp1-in.isites.gff3.gz



In [15]:
# Load UMI count tables produced by TRUmiCount
# Note that TRUmiCount thinks in terms of RNA-Seq and calls whatever key it used
# to group and count reads a "gene". In our case that key is the cassette insertion
# site, which is therefore stored in the column "gene" in TRUmiCount's output.
load.trumicount.output <- function(file, flank) {
  message("Loading TRUmiCount results from ", file)
  t <- data.table(read.table(file, header=TRUE, sep='\t'))
  # Rename column "gene" to "mutant"
  colnames(t) <- ifelse(colnames(t) == "gene", "mutant", colnames(t))
  # Make sure there is exactly one row per mutant and add flank
  allKOs <- data.table(mutant=mutants$Name)
  t <- t[allKOs,, on="mutant"]
  t[, flank := flank ]
}
counts.flank.in.5p <- load.trumicount.output(snakemake@input[["pool_in_5p"]], "5p")
counts.flank.in.3p <- load.trumicount.output(snakemake@input[["pool_in_3p"]], "3p")
counts.flank.out.5p <- load.trumicount.output(snakemake@input[["pool_out_5p"]], "5p")
counts.flank.out.3p <- load.trumicount.output(snakemake@input[["pool_out_3p"]], "3p")

# Combine 5' and 3' data into a single table
counts.flank.in <- rbind(counts.flank.in.5p, counts.flank.in.3p)
counts.flank.out <- rbind(counts.flank.out.5p, counts.flank.out.3p)

# Function to combine flank-specific data horizontally
join.flanks.as.columns <- function(data) {
  data[, list(n5p=n.obs[flank=="5p"],
              l5p=loss[flank=="5p"],
              n3p=n.obs[flank=="3p"],
              l3p=loss[flank=="3p"])
       , by="mutant"]
}


Loading TRUmiCount results from ../data/Transposon.v2/Exp1-in+5p.count.tab

Loading TRUmiCount results from ../data/Transposon.v2/Exp1-in+3p.count.tab

Loading TRUmiCount results from ../data/Transposon.v2/Exp1-out+5p.count.tab

Loading TRUmiCount results from ../data/Transposon.v2/Exp1-out+3p.count.tab



In [16]:
# Aggregate the data for the 5' and 3' flank of each gene.
# We add the counts and average the losses of the two flanks for each gene,.
# We also remember how many flanks were detected for every mutant, so that
# we can later avoid biasing against mutants where some flank is not detecable
# e.g. due to low mapping qualities
aggregate.flanks <- function(counts.flank) {
  counts.flank[, list(
    n = sum(n.obs, na.rm=TRUE),
    flanks = sum(n.obs > 0, na.rm=TRUE),
    loss = mean(loss, na.rm=TRUE)
  ), by="mutant"]
}
counts.in <- aggregate.flanks(counts.flank.in)
counts.out <- aggregate.flanks(counts.flank.out)

# Combine input and output pool data into a single table
counts <- merge(counts.in, counts.out, by="mutant", suffixes=c(".in", ".out"))[, {
  flanks <- pmax(flanks.in, flanks.out, 0, na.rm=TRUE)
  list(mutant,
       is.neutral=mutant %in% NEUTRAL,
       gene=NA_integer_,
       n.in, loss.in,
       n.out, loss.out,
       flanks=flanks,
       abundance.in=ifelse(n.in > 0, n.in / (flanks * (1-loss.in)), 0),
       abundance.out=ifelse(n.out > 0, n.out / (flanks * (1-loss.out)), 0)
)}]


In [27]:
head(counts.in)
head(counts.out)

mutant,n,flanks,loss
<chr>,<int>,<int>,<dbl>
NC_026478.1|Chr01:4003(-),45,2,0.2020
NC_026478.1|Chr01:4216(-),13,2,0.1975
NC_026478.1|Chr01:9179(-),0,0,NaN
NC_026478.1|Chr01:9282(-),0,0,NaN
NC_026478.1|Chr01:9282(+),0,0,NaN
NC_026478.1|Chr01:33155(-),92,2,0.1925


mutant,n,flanks,loss
<chr>,<int>,<int>,<dbl>
NC_026478.1|Chr01:4003(-),3,2,0.3315
NC_026478.1|Chr01:4216(-),14,2,0.3340
NC_026478.1|Chr01:9179(-),0,0,NaN
NC_026478.1|Chr01:9282(-),0,0,NaN
NC_026478.1|Chr01:9282(+),0,0,NaN
NC_026478.1|Chr01:33155(-),75,2,0.3270


In [17]:
counts[, stopifnot(is.na(n.in) == is.na(abundance.in)) ]
counts[, stopifnot(is.na(n.out) == is.na(abundance.out)) ]

NULL

NULL

In [20]:
# Fill in index of the gene (if any) affected by a mutant
mutant_gene <- data.table(mutant=mutants$Name, gene=mutants$GeneIndex)
counts[, "gene" := mutant_gene[counts, gene, on=c("mutant")] ]
head(mutant_gene)

mutant,gene
<chr>,<int>
NC_026478.1|Chr01:4003(-),2
NC_026478.1|Chr01:4216(-),3
NC_026478.1|Chr01:9179(-),NA
NC_026478.1|Chr01:9282(-),NA
NC_026478.1|Chr01:9282(+),NA
NC_026478.1|Chr01:33155(-),NA


In [ ]:
###skipping many of the outputs in the report

## Correlation of Input and Output Abundances

In [22]:
counts[, cor.intercept0(sqrt(n.in), sqrt(n.out))]
counts[, cor.intercept0(sqrt(abundance.in), sqrt(abundance.out))]

Warning message in sqrt(r.sq):
“NaNs produced”


[1] NaN

Warning message in sqrt(r.sq):
“NaNs produced”


[1] NaN

In [39]:
counts[, cor.intercept0(sqrt(n.in), sqrt(n.out))]

[1] 0.3012348

In [52]:
cor.intercept0(c(1,2,3), c(4,5,22))

[1] 0.8175882

In [62]:
my_cor.intercept0 <- function (x, y) 
{
    i <- !is.na(x) & !is.na(y)
    k <- sum(x[i] * y[i])/sum(x[i]^2)
    r.sq <- 1 - sum((y[i] - k * x[i])^2)/sum((y[i] - mean(y[i]))^2)
    r.sq
#    sqrt(r.sq)
}

In [63]:
my_cor.intercept0(
    counts[, sqrt(n.in)],
    counts[, sqrt(n.out)]
    )

[1] -0.01412264

In [ ]:
#so that means that we get "-" if there is in fact negative relationship